In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath("../../"))

In [2]:
from brmspy import brms

In [3]:
data = brms.get_brms_data("epilepsy")

R callback write-console: Error in loadNamespace(x) : there is no package called ‘cmdstanr’
  


cmd CALL


R callback write-console: CmdStan path set to: /Users/sebastian/.brmspy/runtime/macos-arm64-r4.5-0.2.0/cmdstan
  


In [4]:
formula = "count ~ zAge + zBase * Trt + (1|patient)"
family = "poisson"
result = brms.brm(
    formula=formula,
    data=data,
    family=family,
    warmup=500,
    iter=1000,
    chains=4
)

[brmspy][worker_main] Fitting model with brms (backend: cmdstanr)...
R callback write-console: Compiling Stan program...
  


cmd CALL


R callback write-console: Start sampling
  


Running MCMC with 4 chains, at most 2 in parallel...

Chain 1 Iteration:   1 / 1000 [  0%]  (Warmup) 
Chain 2 Iteration:   1 / 1000 [  0%]  (Warmup) 
Chain 1 Iteration: 100 / 1000 [ 10%]  (Warmup) 
Chain 2 Iteration: 100 / 1000 [ 10%]  (Warmup) 
Chain 1 Iteration: 200 / 1000 [ 20%]  (Warmup) 
Chain 2 Iteration: 200 / 1000 [ 20%]  (Warmup) 
Chain 1 Iteration: 300 / 1000 [ 30%]  (Warmup) 
Chain 2 Iteration: 300 / 1000 [ 30%]  (Warmup) 
Chain 1 Iteration: 400 / 1000 [ 40%]  (Warmup) 
Chain 2 Iteration: 400 / 1000 [ 40%]  (Warmup) 
Chain 1 Iteration: 500 / 1000 [ 50%]  (Warmup) 
Chain 1 Iteration: 501 / 1000 [ 50%]  (Sampling) 
Chain 2 Iteration: 500 / 1000 [ 50%]  (Warmup) 
Chain 2 Iteration: 501 / 1000 [ 50%]  (Sampling) 
Chain 1 Iteration: 600 / 1000 [ 60%]  (Sampling) 
Chain 2 Iteration: 600 / 1000 [ 60%]  (Sampling) 
Chain 1 Iteration: 700 / 1000 [ 70%]  (Sampling) 
Chain 2 Iteration: 700 / 1000 [ 70%]  (Sampling) 
Chain 1 Iteration: 800 / 1000 [ 80%]  (Sampling) 
Chain 2 Iteration: 8

In [5]:
brms.summary(result)

cmd CALL


 Family: poisson 
  Links: mu = log 
Formula: count ~ zAge + zBase * Trt + (1 | patient) 
   Data: structure(list(Age = c(31, 30, 25, 36, 22, 29, 31, (Number of observations: 236) 
  Draws: 4 chains, each with iter = 1000; warmup = 500; thin = 1;
         total post-warmup draws = 2000

Multilevel Hyperparameters:
~patient (Number of levels: 59) 
              Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sd(Intercept)     0.58      0.07     0.47     0.74 1.01      466      921

Regression Coefficients:
          Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
Intercept     2.04      0.26     1.53     2.56 1.01      406      744
zAge          0.09      0.09    -0.09     0.26 1.01      415      747
zBase         0.64      0.28     0.09     1.20 1.01      366      595
Trt          -0.27      0.17    -0.60     0.06 1.01      415      779
zBase:Trt     0.06      0.17    -0.29     0.39 1.01      383      535

Draws were sampled using sample(hmc). For each parameter

In [6]:
brms.call("summary", result)

cmd CALL


R callback write-console: Error: 'summary' is not an exported object from 'namespace:brms'
  


{'formula': {'formula': 'count ~ zAge + zBase * Trt + (1 | patient)\nattr(,"nl")\n[1] FALSE\nattr(,"loop")\n[1] TRUE\n<environment: 0x7abf58660>\n',
  'pforms': [],
  'pfix': [],
  'resp': 'count',
  'family': {'family': 'poisson',
   'link': 'log',
   'linkfun': 'function (mu) \nlink(mu, link = slink)\n<bytecode: 0x7ab02aa18>\n<environment: 0x7ac45ad28>\n',
   'linkinv': 'function (eta) \ninv_link(eta, link = slink)\n<bytecode: 0x7ab02a7e8>\n<environment: 0x7ac45ad28>\n',
   'dpars': 'mu',
   'type': 'int',
   'ybounds': [0.0, inf],
   'closed': [True, <brmspy.install_rpy2_stub.<locals>.Obj at 0x11519aae0>],
   'ad': ['weights', 'subset', 'cens', 'trunc', 'rate', 'index'],
   'specials': 'sbi_log'},
  'mecor': True},
 'data_name': 'structure(list(Age = c(31, 30, 25, 36, 22, 29, 31,',
 'group': 'patient',
 'nobs': 236,
 'ngrps': {'patient': 59},
 'autocor': None,
 'prior': Empty DataFrame
 Columns: [prior, class, coef, group, resp, dpar, nlpar, lb, ub, tag, source]
 Index: [],
 'algori

In [7]:
#brms.shutdown()